### Fake News Detection

In [58]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [59]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
df_true = pd.read_csv('True.csv')
df_fake = pd.read_csv('Fake.csv')

In [61]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [62]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [63]:
df_true.shape

(21417, 4)

In [64]:
df_fake.shape

(23481, 4)

In [65]:
df_true["class"] = 1
df_fake["class"] = 0

In [66]:
df_true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [67]:
df_fake.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [68]:
df_news = pd.concat([df_fake, df_true], axis=0)


In [69]:
df_news.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [70]:
df_news.shape

(44898, 5)

In [71]:
df_news = df_news[["text","class"]]

In [72]:
df_news.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [73]:
df_news.shape

(44898, 2)

In [74]:
df_news = df_news.sample(frac=1)
df_news = df_news.reset_index(drop=True)

In [75]:
df_news

,text,class
0,(Reuters) - Transgender recruits will be able ...,1
1,MOSCOW (Reuters) - French President Emmanuel M...,1
2,"Steve Bannon, Trump s campaign CEO who was the...",0
3,If you want to know what kind of President Don...,0
4,"LOUISVILLE, Ky. (Reuters) - Protesters at a Do...",1
...,...,...
44893,WASHINGTON (Reuters) - President Donald Trump ...,1
44894,This ex-ICE agent reminds us of how ridiculous...,0
44895,MSNBC host Lawrence O Donnell did not hold bac...,0
44896,WASHINGTON (Reuters) - A U.S. Senate committee...,1


In [76]:
df_news.isnull().sum()

text     0
class    0
dtype: int64

In [77]:
ps = PorterStemmer()

def stemming(text):
    stemming_text = re.sub('[^a-zA-Z]', ' ', text)
    stemming_text = stemming_text.lower()
    stemming_text = stemming_text.split()
    temp_stemming_text_list = []
    for word in stemming_text:
        if word not in stopwords.words('english'):
            temp_stemming_text_list.append(ps.stem(word))
    stemming_text = ' '.join(temp_stemming_text_list)
    return stemming_text


In [78]:
df_news["text"] = df_news["text"].apply(stemming)

In [79]:
x = df_news["text"].values
y = df_news["class"].values

In [80]:
vector = TfidfVectorizer()
vector.fit(x)
x = vector.transform(x)

In [81]:
x_train, x_test, y_train, y_test = train_test_split(x ,y , test_size=0.3, random_state=42)

In [82]:
print("x_train_shape:", x_train.shape)
print("y_train_shape:", y_train.shape)
print("x_test_shape:", x_test.shape)
print("y_test_shape:", y_test.shape)

x_train_shape: (31428, 89633)
y_train_shape: (31428,)
x_test_shape: (13470, 89633)
y_test_shape: (13470,)


### Create Logistic Regression

In [83]:
model_lr = LogisticRegression()
model_lr.fit(x_train, y_train)

LogisticRegression()

In [84]:
lr_train_pred = model_lr.predict(x_train)
print('Logistic Regression Train Accuracy: ', accuracy_score(lr_train_pred,y_train))

Logistic Regression Train Accuracy:  0.9905498281786942


In [85]:
lr_test_pred = model_lr.predict(x_test)
print("Logistic Regression Test Accuracy: ", accuracy_score(lr_test_pred, y_test))

Logistic Regression Test Accuracy:  0.9847067557535264


### Prediction

In [86]:
text = input("Enter the text: ")
text = stemming(text)
text = vector.transform([text])
text_pred = model_lr.predict(text)
if text_pred == 0:
    print("Fake News")
else:
    print("Real News")

Enter the text: House Intelligence Committee Chairman Devin Nu...'
Fake News


In [ ]:
import joblib
from google.colab import files
# Save the model to a file
joblib_file = "model.pkl"
joblib.dump(model_lr, joblib_file)

# Download the model file
files.download(joblib_file)

In [90]:
# Save the vectorizer
joblib.dump(vector, 'vectorizer.pkl')
files.download('vectorizer.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>